In [ ]:
#make sure to install entropica on device

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('QAOA/data/leagueoflegends.csv')#file path of csv
df = df.set_index('#') #index character by their ID number
df = df.rename_axis('ID') #rename axis to 'ID' instead of '#'
df = df.loc[~df.index.duplicated(keep='first')] #drop duplicates
df.head()

df = df.loc[df['Generation']<=3]
df.sample(frac=1).head() #sample the whole dataset (frac=1) to shuffle the arrangement

print('Percent of Non-Legendary Champions: %.2f' %((df.Legendary.count()-df.Legendary.sum())/df.Legendary.count()))
print('Percent of Legendary Champions: %.2f' %((df.Legendary.sum())/df.Legendary.count()))

legendary = df.loc[df['Legendary'] == True].sample(5)
non_legendary = df.loc[df['Legendary'] == False].sample(5)
league = pd.concat([legendary,non_legendary])

numerical_columns = ['Total','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']

labels = league['Legendary']
data = league[numerical_columns].copy()
data.head()

from entropica_qaoa.utilities import distances_dataset

dist = pd.DataFrame(distances_dataset(data.values),
                       index=data.index,columns=data.index)
dist.iloc[0:5, 0:5]

df.loc[dist.index].head()


PYQUIL Portion

In [ ]:
from pyquil.api import WavefunctionSimulator
from pyquil.paulis import PauliSum, PauliTerm

#Constructing a quadratic PauliTerm
i = 3
j = 6
print('Distance between samples %d and %d: %.3f' %(i,j,dist.values[i][j]))

term1 = PauliTerm("Z",i,dist.values[i][j])
term2 = PauliTerm("Z",j,1.0) 
term = term1*term2
print(term)

from entropica_qaoa.utilities import hamiltonian_from_distances

hamiltonian = hamiltonian_from_distances(dist)
print(hamiltonian)

# import the neccesary pyquil modules
from entropica_qaoa.qaoa.cost_function import QAOACostFunctionOnQVM, QAOACostFunctionOnWFSim

# import QAOAParameters 
from entropica_qaoa.qaoa.parameters import ExtendedParams

# import an optimizer
from scipy.optimize import minimize

#Some utilities for time tracking and measuring our outcomes.
import time
from math import log
from entropica_qaoa.utilities import cluster_accuracy, max_probability_bitstring


timesteps = 3 # The QAOA p parameter
iters = 500 # Number of classical optimiser iterations
n_qubits = 10 #this number might be defined before your dataset - should equal the number of data points
#The hamiltonian is also a hyperparameter

betas = [round(val,1) for val in np.random.rand(timesteps*n_qubits)]
gammas_singles = [round(val,1) for val in np.random.rand(0)] #we don't want any bias terms
gammas_pairs = [round(val,1) for val in np.random.rand(timesteps*len(hamiltonian))]

hyperparameters = (hamiltonian, timesteps)
parameters = (betas, gammas_singles, gammas_pairs)

params = ExtendedParams(hyperparameters, parameters)

# Set up the WavefunctionSimulator from pyQuil
sim = WavefunctionSimulator()
cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                        params=params,
                                        sim=sim,
                                        enable_logging=True)

t0 = time.time()
res = minimize(cost_function, params.raw(), tol=1e-3, method='Cobyla',
               options={"maxiter": iters})
print('Run complete!\n','Runtime:','{:.3f}'.format(time.time()-t0))

wave_func = cost_function.get_wavefunction(params.raw())
lowest = max_probability_bitstring(wave_func.probabilities())

true_clusters = [1 if val else 0 for val in labels] 
acc = cluster_accuracy(lowest,true_clusters)

print('Cost Function Value:', res.fun)
print('Converged?:',res.message)

